In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
from torchvision.transforms import transforms

from lib.path import data_file_path, model_file_path, output_path


In [4]:
data_transform = transforms.Compose(
    # fmt : off
    [
        transforms.ToTensor(),
        transforms.Resize(32),
        transforms.Normalize((0.5,), (1.0,)),  # 간이 정규화
    ]
    # fmt : on
)

train_data = datasets.MNIST(
    root=data_file_path(), train=True, download=True, transform=data_transform
)
test_data = datasets.MNIST(
    root=data_file_path(), train=False, download=True, transform=data_transform
)


In [5]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)


In [6]:
next(iter(train_loader))[0].shape

torch.Size([32, 1, 32, 32])

In [7]:
class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(
            in_channels=16, out_channels=120, kernel_size=5, stride=1
        )
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv3(x)
        x = F.tanh(x)
        x = x.view(-1, 120)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.tanh(x)

        return x


model = Lenet()
model

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
list(model.parameters())

[Parameter containing:
 tensor([[[[ 0.1098, -0.1839,  0.1087, -0.0049,  0.0470],
           [ 0.1576,  0.1310,  0.1260, -0.0651,  0.1801],
           [-0.1834, -0.0769, -0.1657, -0.1580,  0.1723],
           [ 0.0582, -0.1335,  0.0332,  0.1615,  0.1252],
           [-0.1984,  0.0308,  0.1469,  0.1814,  0.0028]]],
 
 
         [[[-0.1792,  0.0246,  0.1720,  0.0941, -0.0209],
           [-0.1731, -0.1220, -0.0147, -0.1410,  0.0933],
           [-0.1597, -0.0524,  0.1183, -0.1285,  0.0238],
           [ 0.1638,  0.0096,  0.1418, -0.0791, -0.1194],
           [-0.0207, -0.1753,  0.1449,  0.1182, -0.0920]]],
 
 
         [[[-0.0959,  0.1199,  0.0366, -0.1885,  0.1113],
           [ 0.0203, -0.1247, -0.1154, -0.1239,  0.1888],
           [ 0.1634,  0.0908, -0.0686, -0.0843, -0.0959],
           [ 0.1313,  0.0800,  0.0020,  0.0628, -0.1591],
           [-0.1517, -0.0667,  0.0944, -0.1426, -0.0919]]],
 
 
         [[[-0.0270, -0.1587, -0.0056, -0.0832, -0.1920],
           [ 0.1138, -0.0784, -

In [9]:
model(torch.randn(1, 1, 32, 32))

tensor([[ 0.0103,  0.0904,  0.0171,  0.1009,  0.0721, -0.0756,  0.0834,  0.1318,
          0.1025, -0.1266]], grad_fn=<TanhBackward0>)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [11]:
from torchsummary import summary

summary(model, input_size=(1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Conv2d-3            [-1, 120, 1, 1]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


In [ ]:
import tqdm
from torch.utils.tensorboard import SummaryWriter

log_dir = output_path() / 'tensorboard'
writer = SummaryWriter(log_dir=log_dir)

optim = Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
epochs = 20

count = 0

# 체크포인트 불러오기


for ep in range(epochs):
    train_tqdm = tqdm.tqdm(train_loader)
    for data, label in train_tqdm:
        data = data.to(device)
        label = label.to(device)
        optim.zero_grad()
        pred = model(data)
        loss = criterion(pred, label)
        writer.add_scalar('Loss/train', loss, count)
        count += 1
        loss.backward()
        optim.step()

        train_tqdm.set_description(f'epoch : {ep + 1} loss : {loss.item():.2f}')
    # 체크포인트 저장

In [ ]:
model.eval()

with torch.no_grad():
    total_corr = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        preds = model(images)

        _, pred = torch.max(preds, 1)
        total_corr += (pred == labels).sum().item()

print(f'Acc {total_corr / len(test_data.targets)}')

In [ ]:
labels

In [ ]:
# 가중치만 저장
torch.save(model.state_dict(), model_file_path('lenet_cnn.pth'))

In [ ]:
model_new = Lenet()
model_new.load_state_dict(torch.load(model_file_path('lenet_cnn.pth')))

In [ ]:
# 모델을 통째로 저장
torch.save(model, model_file_path('lenet_model.pth'))

In [ ]:
# 체크 포인트 저장
torch.save(
    {
        'epoch': ep,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'loss': loss,
    },
    model_file_path('checkpoint.pth'),
)

checkpoint = torch.load('checkpoint.pth')

model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
ep = checkpoint['epoch']
loss = checkpoint['loss']

NameError: name 'torch' is not defined

In [ ]:
# 추론(infer)

from PIL import Image

img = Image.open(data_file_path('4.png'))
img.size

In [ ]:
import numpy as np

img_np = np.array(img)
img_np.shape

In [ ]:
# 학습할때와 같은 전처리
infer_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Grayscale(),
        transforms.Resize(32),
        transforms.Normalize((0.5,), (1.0,)),  # 간이 정규화
    ]
)

# img = infer_transform(img_np)
img = infer_transform(img)
img.size()


In [ ]:
pred = model(img.to(device))
torch.max(pred, 1)[1].item()